In [169]:
# Dependencies and Libraries
from splinter import Browser
from bs4 import BeautifulSoup as bs
import time
from webdriver_manager.chrome import ChromeDriverManager
from pprint import pprint
import pandas as pd
import requests
import regex as re




In [170]:
# Set up Splinter
# Choose the executable path to driver for MAC users
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 91.0.4472
Get LATEST driver version for 91.0.4472
Driver [/Users/yazminrodriguez/.wdm/drivers/chromedriver/mac64/91.0.4472.101/chromedriver] found in cache


### NASA Mars News
Finding the title and paragraph from: https://redplanetscience.com/

In [171]:
# Visit visitcostarica.herokuapp.com
url = "https://redplanetscience.com/"
browser.visit(url)


In [172]:
 # Scrape page into Soup
html = browser.html
soup = bs(html, "html.parser")

# Get the news title and news paragraph
news_title = soup.find('div',class_='content_title').text

# Paragraph
parag= soup.find('div',class_='article_teaser_body').text

In [173]:
# Save title and paragraph in dictionary mars
mars_info={}
mars_info["news_title"]= news_title
mars_info["news_paragraph"]= parag

In [174]:
#Print dict
mars_info

{'news_title': 'Scientists Explore Outback as Testbed for Mars ',
 'news_paragraph': "Australia provides a great place for NASA's Mars 2020 and the ESA-Roscosmos ExoMars scientists to hone techniques in preparation for searching for signs ancient life on Mars."}

### JPL Mars Space Images - Featured Image
Finding images from: https://spaceimages-mars.com/

In [175]:
# Visit visitcostarica.herokuapp.com
url =  'https://spaceimages-mars.com/'
browser.visit(url)

# Find and click the full image button
full_image= browser.find_by_tag('button')[1]
full_image.click()

 # Scrape page into Soup
html_image = browser.html
soup = bs(html_image, "html.parser")

In [176]:
# JPL Mars Space Images - Featured Image
relative_image_path = soup.find('img', class_='fancybox-image').get('src')

#Joining the URL of the page + the URL of the image
featured_image_url= url + relative_image_path

#Adding the image to Directory Mars
mars_info['mars_images']=featured_image_url

# Save image feature in dictionary mars
mars_info["featured_image"]= featured_image_url


In [177]:
#Printing the directory
mars_info

{'news_title': 'Scientists Explore Outback as Testbed for Mars ',
 'news_paragraph': "Australia provides a great place for NASA's Mars 2020 and the ESA-Roscosmos ExoMars scientists to hone techniques in preparation for searching for signs ancient life on Mars.",
 'mars_images': 'https://spaceimages-mars.com/image/featured/mars1.jpg',
 'featured_image': 'https://spaceimages-mars.com/image/featured/mars1.jpg'}

### Mars Facts
Finding a table from: https://galaxyfacts-mars.com/

In [178]:
# Visit visitcostarica.herokuapp.com
url =  'https://galaxyfacts-mars.com/'
browser.visit(url)

 # Scrape page into Soup
html = browser.html
soup = bs(html, "html.parser")

In [179]:
# Use Panda's read_html to parse the url
mars_facts_table = pd.read_html(url, match='Equatorial Diameter')

# Find the mars facts DataFrame in the list of DataFrames and assign it to mars_df
mars_df = mars_facts_table[0]

# Assign the columns Category, Value
mars_df.columns = ['Category','Value']

# Set the index to the 'Category' column without row indexing
mars_df.set_index('Category', inplace=True)

# Save html code to folder Assets
mars_df.to_html()

data = mars_df.to_dict(orient='records')

# Save Data
mars_info['facts_table']= mars_df

In [180]:
#Printing dict
mars_df

,Value
Category,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.39 × 10^23 kg (0.11 Earths)
Moons:,2 ( Phobos & Deimos )
Orbit Distance:,"227,943,824 km (1.38 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-87 to -5 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


### Mars Hemispheres

Obtaining high resolution images for each of Mar's hemispheres.https://marshemispheres.com/


In [181]:
# Visit the website

url =  'https://marshemispheres.com/'
browser.visit(url)
# Html Object
html = browser.html 

# Parse HTML with Beautiful Soup
soup=bs(html, 'html.parser')

In [182]:

# Create a list for the full images url's
hemispheres_img_urls = []

# Retrieve all items that contain mars hemispheres inform action
items = browser.find_by_css('a.product-item img')

## Looping throught the links (items), click the link, find the sample anchor
# return the href


for x in range(len(items)):
    
    hemispheres={}
    
    full_image_item = browser.find_by_tag('h3')[x]
    full_image_item.click()
    html= browser.html
    img_soup= bs(html, 'html.parser')
    # Finding the samplae image anchor tag and extact the href
    img= img_soup.find('a', text= 'Sample').attrs['href']
    full_img = f'{url}{img}'
    
    
    # Get Hemisphere title
    title = img_soup.find('h2', class_='title').get_text()
    # Append hemisphere object to list
    hemispheres['title'] = title
    hemispheres['img_url'] = full_img
    hemispheres_img_urls.append(hemispheres)
    
    # We need to go back
    browser.back()
    

In [183]:
print(hemispheres_img_urls)

[{'title': 'Cerberus Hemisphere Enhanced', 'img_url': 'https://marshemispheres.com/images/full.jpg'}, {'title': 'Schiaparelli Hemisphere Enhanced', 'img_url': 'https://marshemispheres.com/images/schiaparelli_enhanced-full.jpg'}, {'title': 'Syrtis Major Hemisphere Enhanced', 'img_url': 'https://marshemispheres.com/images/syrtis_major_enhanced-full.jpg'}, {'title': 'Valles Marineris Hemisphere Enhanced', 'img_url': 'https://marshemispheres.com/images/valles_marineris_enhanced-full.jpg'}]


In [184]:
mars_info['hemispheres_img']= hemispheres_img_urls
mars_info

{'news_title': 'Scientists Explore Outback as Testbed for Mars ',
 'news_paragraph': "Australia provides a great place for NASA's Mars 2020 and the ESA-Roscosmos ExoMars scientists to hone techniques in preparation for searching for signs ancient life on Mars.",
 'mars_images': 'https://spaceimages-mars.com/image/featured/mars1.jpg',
 'featured_image': 'https://spaceimages-mars.com/image/featured/mars1.jpg',
 'facts_table':                                               Value
 Category                                           
 Equatorial Diameter:                       6,792 km
 Polar Diameter:                            6,752 km
 Mass:                 6.39 × 10^23 kg (0.11 Earths)
 Moons:                        2 ( Phobos & Deimos )
 Orbit Distance:            227,943,824 km (1.38 AU)
 Orbit Period:                  687 days (1.9 years)
 Surface Temperature:                   -87 to -5 °C
 First Record:                     2nd millennium BC
 Recorded By:                   Egyptian as

In [185]:
# Quit the browser
browser.quit()